<a href="https://colab.research.google.com/github/FIJY/hh/blob/master/Copy_of_%D0%9F%D0%B0%D1%80%D1%81%D0%B8%D0%BD%D0%B3_hh_ver3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Парсинг с сайта hh <br>
Ссылка на видео парсинга hh https://yandex.ru/video/preview/11468178118806870731

## Устанавливаем и импортируем библиотеки

In [72]:
# Устанавливаем библиотеки
# requests для отправки запросов
# BeautifulSoup для разбора полученных данных
# lxml для парсинга
# fake_useragent для создания заголовков
# !pip install requests (requests в colab можно не устанавливать)
!pip install bs4
# !pip install lxml (lxml в colab можно не устанавливать)
!pip install fake_useragent



In [73]:
# Импортируем библиотеки
import requests
from bs4 import BeautifulSoup
import fake_useragent
import time
import json
import pandas as pd
import re
import csv



## Парсинг hh

In [ ]:
# Заходим на hh, выбираем регион (Россия)
# Вводим запрос в поле поиска (например python) в разделе Соискателям
# Переходим на вкладку резюме
# Переходим на вторую страницу, чтобы появился параметр page
# Копируем адрес страницы

### Создание параметров поиска резюме, <br>получение ссылки-запроса для последующего парсинга ссылок резюме.  <br><br> Источник параметров поиска <br> https://github.com/EvgenyTerebenin/hh-api/blob/master/docs/resumes_search.md

In [48]:
# Функция создания ссылки для поиска резюме по заданным параметрам

def ref_search_resume (text: str, what_search = 'normal', list_search = ['full_text'],
                       sorting = 'relevance', date_period = 0, date_min = None,
                       date_max = None, quantity_items = 20, areas = [113],
                       arg_relocation = "living_or_relocation", exp_industry = None,
                       exp_period = 'all_time', job_search = None, age_min = None,
                       age_max = None, age_if = False, employments = None,
                       schedules = None, experiences = None, prof_roles = None,
                       gen = "unknown", gender_if = False, salary_min = None,
                       salary_max = None, currency = "RUB", salary_if = False,
                       countries = None, foto_if = False, vehicle_if = False,
                       educations = None, universities = None, ver_skills = None,
                       key_skills = None, work_countries = None,
                       driver_types = None, languages = None,
                       verif_languages = None):

  '''
  Функция создания ссылки для поиска резюме по заданным параметрам.

  :text: Слово или фраза для поиска резюме.

  :what_search = 'normal': Какие слова использовать в поиске
  "Все слова": normal,
  "Любое из слов": any,
  "Точная фраза": phrase,
  "Не встречаются": except

  :list_search = ['full_text']: Где искать указанные слова
  "Везде": full_text,
  "В названии резюме": position,
  "В образовании": education,
  "В ключевых навыках": keywords,
  "В опыте работы": workplaces
  При уточнении можно подавать несколько, списком, но не более трех,
  иначе будет вариант "Везде".

  :sorting = 'relevance': Способ сортировки списка резюме.
  "По дате изменения": publication_time,
  "По убыванию зарплат": salary_desc,
  "По возрастанию зарплаты": salary_asc,
  "По соответствию": relevance

  :date_period = 0: За какой период искать
  "За все время": 0,
  "За сутки": 1,
  "За последние три дня": 3,
  "За неделю": 7,
  "За месяц": 30,
  "За год": 365

  :date_min = None  date_max = None: Интервал поиска между датами
  Формат указания дат ДД.ММ.ГГГГ

  :quantity_items = 20: Количество резюме на одной странице
  Можно указывать любое количество. Стандартный выбор: 20, 50, 70, 100

  :areas = [113]: Регион поиска
  Можно выбирать несколько, подается список кодов регионов
  Источник https://api.hh.ru/areas
  Обозначения некоторых регионов:
  Россия: 113,
  Москва: 1, Московская область: 2019,
  Санкт-Петербург: 2, Ленинградская область: 145,
  Екатеринбург: 3, Свердловская область: 1261,
  Новосибирск: 4, Новосибирская область: 1202,
  Нижний Новгород: 66, Красноярск: 54, Челябинск: 104, Самара: 78, Уфа: 99,
  Краснодар: 53, Ростов-на-Дону: 76, Омск: 68, Воронеж: 26, Пермь: 72, Волгоград: 24,
  Татарстан: 1624, Казань: 88

  :arg_relocation = "living_or_relocation": Переезд
  Источник
  https://api.hh.ru/openapi/redoc#tag/Obshie-spravochniki/operation/get-dictionaries
  Варианты, подается один из вариантов:
  "living_or_relocation" - Живут в указанном регионе или готовы переехать в него,
  "living" - Живут в указанном регионе,
  "living_but_relocation" - Живут в указанном регионе и готовы к переезду куда-либо,
  "relocation" - "Не живут в указанном регионе, но готовы переехать в него"

  :exp_industry = None: Опыт работы в отрасли
  Указывается требуемая отрасль;
  По умолчанию выбраны все отрасли
  Можно выбирать несколько, подается списком кодов отраслей
  Источник https://api.hh.ru/industries
  "Информационные технологии, системная интеграция, интернет": "7",
  включает в себя:
  "Интернет-провайдер": "7.538",
  "Системная интеграция,  автоматизации технологических и
  бизнес-процессов предприятия, ИТ-консалтинг": "7.539",
  "Разработка программного обеспечения": "7.540",
  "Интернет-компания (поисковики, платежные системы, соц.сети,
  информационно-познавательные и развлекательные ресурсы,
  продвижение сайтов и прочее)": "7.541"

  :exp_period = 'all_time': Период работы в отрасли, указывается вместе с опытом работы
  Источник
  https://api.hh.ru/openapi/redoc#tag/Poisk-rezyume/operation/search-for-resumes
  Варианты, выбирается один из вариантов:
  "all_time" - за всё время работы;
  "last_year" - за последний год;
  "last_three_years" - за последние 3 года;
  "last_six_years" - за последние 6 лет

  :job_search = None: Статус поиска работы
  Варианты, можно выбирать несколько, подается списком:
  "unknown" - Без статуса поиска;
  "not_looking_for_job" - Не ищет работу;
  "looking_for_offers" - Рассматривает предложения;
  "active_search" - Активно ищет работу;
  "has_job_offer" - Предложили работу, решает;
  "accepted_job_offer" - Вышел на новое место

  :age_min = None  age_max = None: Возраст, от и до
  :age_if = False: Найти резюме только с возрастом
  (по умолчанию фильтр отсутствует)

  :employments = None: Тип занятости
  По умолчанию не указан
  Варианты, можно выбирать несколько, подается списком:
  "full" - Полная занятость;
  "part" - Частичная занятость;
  "project" - Проектная работа;
  "probation" - Стажировка;
  "volunteer" - Волонтерство

  :schedules = None: График работы
  По умолчанию не указан
  Варианты, можно выбирать несколько, подается списком:
  "fullDay" - Полный день;
  "remote" - Удаленная работа;
  "flexible" - Гибкий график;
  "shift" - Сменный график;
  "flyInFlyOut" - Вахтовый метод

  :experiences = None: Опыт работы
  По умолчанию не указан
  Варианты, можно выбирать несколько, подается списком:
  "moreThan6" - Более 6 лет;
  "between3And6" - От 3 до 6 лет;
  "noExperience" - Нет опыта;
  "between1And3" - От 1 года до 3 лет

  :prof_roles = None: Специализации
  По умолчанию не указаны
  Перечень кодов специализаций (направление "Информационные технологии"),
  можно выбирать несколько, подается список кодов:
  Источник https://api.hh.ru/professional_roles
  "BI-аналитик, аналитик данных": 156,
  "DevOps-инженер": 160,
  "Аналитик": 10,
  "Арт-директор, креативный директор": 12,
  "Бизнес-аналитик": 150,
  "Гейм-дизайнер": 25,
  "Дата-сайентист": 165,
  "Дизайнер, художник": 34,
  "Директор по информационным технологиям (CIO)": 36,
  "Менеджер продукта": 73,
  "Методолог": 155,
  "Программист, разработчик": 96,
  "Продуктовый аналитик": 164,
  "Руководитель группы разработки": 104,
  "Руководитель отдела аналитики": 157,
  "Руководитель проектов": 107,
  "Сетевой инженер": 112,
  "Системный администратор": 113,
  "Системный аналитик": 148,
  "Системный инженер": 114,
  "Специалист по информационной безопасности": 116,
  "Специалист технической поддержки": 121,
  "Тестировщик": 124,
  "Технический директор (CTO)": 125,
  "Технический писатель": 126,
  "Другое": 40

  :gen = "unknown": Пол
  "unknown" - не имеет значения;
  "male" - мужской;
  "female" - женский

  :gender_if = False: Показать резюме с указанием пола
  По умолчанию фильтр отсутствует

  :salary_min = None: Зарплата от
  :salary_max = None: Зарплата до
  :currency = "RUB": Валюта
  Можно задать в евро EUR или в долларах USD

  :salary_if = False: Показать резюме только где указан доход
  По умолчанию фильтр отсутствует

  :countries = None: Гражданство, страна
  Перечень некоторых кодов стран, можно выбирать несколько, подается список кодов:
  Источник https://api.hh.ru/areas/countries
  Россия: 113,
  Беларусь: 16,
  Китай: 50,
  Греция: 213,
  Другое: 6868

  :foto_if = False  vehicle_if = False: Есть фото, автомобиль
  По умолчанию фильтры отсутствуют

  :educations = None: Образование
  Перечень видов образования, можно выбирать несколько, подается список названий:
  "Среднее": secondary,
  "Среднее специальное": special_secondary,
  "Незаконченное высшее": unfinished_higher,
  "Бакалавр": bachelor,
  "Магистр": master,
  "Высшее": higher,
  "Кандидат наук": candidate,
  "Доктор наук": doctor

  :universities = None: Учебное заведение
  Перечень кодов некоторых учебных заведений, можно выбирать несколько,
  подается список кодов:
  Источник https://app.psal.ru/api/university/
  "МГУ": 39144,
  "Национальный исследовательский университет ИТМО": 39872,
  "НИЯУ МИФИ": 39650,
  "МГТУ им. Н.Э. Баумана": 38921

  :ver_skills = None: Подтвержденные навыки
  Перечень кодов навыков, можно выбирать несколько, подается список кодов:
  "Python": 1114,
  "MS Excel": 845,
  "SQL": 1252,
  "Git": 3215,
  "HTML": 598,
  "ООП": 2350,
  "JavaScript": 674,
  "Agile Project Management": 998,
  "Scrum": 3193,
  "Копирайтинг": 2097,
  "AutoCAD": 153

  :key_skills = None: Ключевые навыки
  Перечень кодов навыков, можно выбирать несколько, подается список кодов:
  'Python': 1114, 'SQL': 1252, 'Git': 3215, 'Linux': 730, 'Английский язык': 5488,
  'HTML': 598, 'MS Excel': 845, 'JavaScript': 674, 'Работа в команде': 4118,
  'MySQL': 893, 'PostgreSQL': 1063, 'MS PowerPoint': 866, 'C++': 231, 'CSS': 351,
  'Управление проектами': 2924, 'ООП': 2350, 'Пользователь ПК': 3864,
  'Adobe Photoshop': 87, 'Java': 3093, 'Организаторские навыки': 2366,
  'Грамотная речь': 1827, 'C#': 230, 'HTML5': 7161, 'Ведение переговоров': 1691,
  'PHP': 3750, 'Работа с большим объемом информации': 6958, 'C/C++': 144745,
  'MS Word': 883, 'Деловая переписка': 1855, 'Django Framework': 416,
  'Atlassian Jira': 142, 'Анализ данных': 1534, 'MS SQL': 871,
  'Аналитическое мышление': 14886, 'MATLAB': 791, 'Bash': 4070, 'CSS3': 3512,
  'Деловое общение': 1857, 'Математическая статистика': 2190, 'MS Outlook': 860,
  'AutoCAD': 153, 'Data Analysis': 372, 'jQuery': 687,
  'Управление персоналом': 2917, 'Scrum': 3193, 'Базы данных': 8540,
  'Работа с базами данных': 17049, 'Проведение презентаций': 2514,
  'Деловая коммуникация': 1853, 'Agile Project Management': 99

  :work_countries = None: Разрешение на работу, страна
  Источник https://api.hh.ru/areas/countries
  Россия: 113,
  Беларусь: 16,
  Китай: 50,
  Греция: 213,
  Другое: 6868

  :driver_types = None: Категория прав
  Перечень категорий, можно выбирать несколько, подается список категорий:
  B, C, A, BE, D, E, CE, DE, Tm (трамвай), Tb (троллейбус)

  :languages = None: Знание языков, уровень
  Можно выбирать несколько, подается список языков
  Перечень некоторых языков:
  Источник https://api.hh.ru/languages
  "Русский": rus, 34;
  "Английский": eng, 57;
  "Французский": fra, 59;
  "Немецкий": deu, 58;
  "Белорусский": bel, 7;
  "Китайский": zho, 20
  С языком указывается уровень владения, например "eng.a1"
  Перечень уровней владения языком:
  a1 - Начальный, a2 - Элементарный,
  b1 - Средний, b2 - Средне-продвинутый,
  c1 - Продвинутый, c2 - В совершенстве,
  l1 - Родной язык

  :verif_languages = None: Подтвержденные знания
  (указываются только по английскому языку)
  Перечень кодов подтвержденных знаний:
  "a1 Начальный": 57.1;
  "a2 Элементарный": 57.2;
  "b1 Средний": 57.3;
  "b2 Средне-продвинутый": 57.4;
  "c1 Продвинутый": 57.5;
  "c2 В совершенстве": 57.6
  '''
  # Слово/фраза для поиска резюме
  # Перед подачей в url заменим в text пробелы и другие символы (,;) на url-коды
  text = text.replace(" ", "+").replace(",", "%2C").replace(";", "%3B")

  # Какие слова использовать в поиске
  # what_search = 'any'
  # what_search = 'phrase'
  # what_search = 'except'
  logic = f'&logic={what_search}'

  # Где искать указанные слова
  # list_search = ['keywords', 'position']
  # list_search = ['keywords', 'position', 'education', 'workplaces']
  # list_search = ['keywords']
  if (len(list_search) < 4):
    where_search = '%2C'.join(list_search)
  else:
    where_search = 'full_text'
  pos = f'&pos={where_search}'

  # Способ сортировки списка резюме
  # sorting = 'salary_desc'
  order_by = f'&order_by={sorting}'

  # За какой период искать
  date_period = 30

  # Интервал поиска между датами
  # date_min = '01.03.2024'
  # date_max = '15.03.2024'
  if (date_min and date_max):
    search_period = f'&search_period=-1&date_from={date_min}&date_to={date_max}'
  else:
    search_period = f'&search_period={date_period}'

  # Количество резюме на одной странице
  # quantity_items = 5
  items_on_page = f'&items_on_page={quantity_items}'

  # Регион поиска
  # areas = [113, 4]
  area = ''.join(f'&area={region}' for region in areas)

  # Переезд
  # arg_relocation = "living"
  relocation = f'&relocation={arg_relocation}'

  # Опыт работы в отрасли
  # exp_industry = [7] # Подается списком кодов отраслей
  # exp_industry = [7.538, 7.540, 7.541]
  filter_exp_industry = ''
  if exp_industry:
    filter_exp_industry = ''.join(f'&filter_exp_industry={exp_industry:.3f}'
                                  if isinstance(exp_industry, float)
                                  else f'&filter_exp_industry={int(exp_industry)}'
                                  for exp_industry in exp_industry)

  # Период опыта работы в отрасли
  # exp_period = "last_year"
  filter_exp_period = f'&filter_exp_period={exp_period}'

  # Статус поиска работы
  # job_search = ["looking_for_offers", "has_job_offer"]
  job_search_status = ''
  if job_search:
    job_search_status = ''.join(f'&job_search_status={search}'
                                for search in job_search)

  # Возраст
  # age_min = 35
  # age_max = 45
  # age_if = True
  age_from = ''
  age_to = ''
  label_age = ''
  if age_min: age_from = f'&age_from={age_min}'
  if age_max: age_to = f'&age_to={age_max}'
  if age_if: label_age = f'&label=only_with_age'

  # Тип занятости
  # employments = ['part', 'probation']
  employment = ''
  if employments:
    employment = ''.join(f'&employment={empl}' for empl in employments)

  # График работы
  # schedules = ['remote', 'flexible']
  schedule = ''
  if schedules:
    schedule = ''.join(f'&schedule={sched}' for sched in schedules)

  # Опыт работы
  # experiences = ['between1And3']
  experience = ''
  if experiences:
    experience = ''.join(f'&experience={exper}' for exper in experiences)

  # Специализации
  # prof_roles = [96, 40, 156]
  professional_role = ''
  if prof_roles:
    professional_role = ''.join(f'&professional_role={prof_role}'
                                for prof_role in prof_roles)

  # Пол
  # gen = "male"
  # gen = "female"
  # gender_if = True
  gender = f'&gender={gen}'
  label_gender = ''
  if gender_if: label_gender = f'&label=only_with_gender'

  # Уровень дохода
  # salary_min = 500
  # salary_max = 1000
  # currency = 'EUR'
  # currency = 'USD'
  # salary_if = True
  salary_from = ''
  salary_to = ''
  label_salary = ''
  currency_code = ''
  if currency in ['EUR', 'USD']: currency_code = f'&currency_code={currency}'
  if salary_min: salary_from = f'&salary_from={salary_min}'
  if salary_max: salary_to = f'&salary_to={salary_max}'
  if salary_if: label_salary = f'&label=only_with_salary'

  # Гражданство, страна
  # countries = [113, 16]
  citizenship = ''
  if countries:
    citizenship = ''.join(f'&citizenship={country}' for country in countries)

  # Есть фото, есть  автомобиль
  # foto_if = True
  # vehicle_if = True
  label_foto = ''
  label_vehicle = ''
  if foto_if: label_foto = f'&label=only_with_photo'
  if vehicle_if: label_vehicle = f'&label=only_with_vehicle'

  # Образование
  # educations = ['higher', 'candidate']
  education_level = ''
  if educations:
    education_level = ''.join(f'&education_level={education}'
                              for education in educations)

  # Учебное заведение
  # universities = [39872, 39650]
  university = ''
  if universities:
    university = ''.join(f'&university={univer}' for univer in universities)

  # Подтвержденные навыки
  # ver_skills = [1114, 845, 2350]
  verified_skills = ''
  if ver_skills:
    verified_skills = ''.join(f'&verified_skills={verif_skill}'
                              for verif_skill in ver_skills)

  # Ключевые навыки
  # key_skills = [3093, 6958, 14886, 1114]
  skill = ''
  if key_skills:
    skill = ''.join(f'&skill={key_skill}' for key_skill in key_skills)

  # Разрешение на работу, страна
  # work_countries = [16, 213]
  work_ticket = ''
  if work_countries:
    work_ticket = ''.join(f'&work_ticket={work_country}'
                          for work_country in work_countries)

  # Категория прав
  # driver_types = ['B', 'C']
  driver_license_types = ''
  if driver_types:
    driver_license_types = ''.join(f'&driver_license_types={type_drive}'
                                   for type_drive in driver_types)

  # Знание языков, уровень
  # Дополнительно по английскому языку можно указать подтвержденные знания
  # languages = ['rus.l1', 'bel.c2', 'eng.c1', 'zho.b1']
  # verif_languages = '57.5'
  language = ''
  if languages:
    language = ''.join(f'&language={lang}' for lang in languages)
  verified_languages = ''
  if languages:
    verified_languages = f'&verified_languages={verif_languages}'

  # Создаем ссылку для поиска резюме по заданным параметрам
  base_url=\
  f"https://hh.ru/search/resume?text={text}{area}&isDefaultArea=true\
&exp_period=all_time{logic}{pos}{order_by}{search_period}{items_on_page}{relocation}\
{filter_exp_industry}{filter_exp_period}{job_search_status}{age_from}{age_to}\
{label_age}{employment}{schedule}{experience}{professional_role}{gender}\
{label_gender}{currency_code}{salary_from}{salary_to}{label_salary}\
{citizenship}{label_foto}{label_vehicle}{education_level}{university}\
{verified_skills}{skill}{work_ticket}{driver_license_types}{language}\
{verified_languages}"

  return base_url

### Парсинг ссылок резюме

In [49]:
# Получаем ссылку-запрос для последующего парсинга ссылок резюме

# Для примера взято ключевое слово для поиска "python"
# и статус соискателя "рассматривает предложения / активно ищет работу"
search = "MS Excel"
base_url = ref_search_resume(search)
base_url

'https://hh.ru/search/resume?text=MS+Excel&area=113&isDefaultArea=true&exp_period=all_time&logic=normal&pos=full_text&order_by=relevance&search_period=30&items_on_page=20&relocation=living_or_relocation&filter_exp_period=all_time&gender=unknown'

In [50]:
# # Проверим работу функции
# base_url = ref_search_resume('python')

# base_url

In [51]:
# Создаем объект для передачи заголовка
# Создадим юзер-агента, общего для всех запросов
ua = fake_useragent.UserAgent()

In [52]:
# Функция для получения ссылок по запросу
def get_links(base_url):

  data = requests.get(
      url=base_url,
      headers = {"user-agent":ua.random})

  # Если произошла ошибка, выходим из функции
  if data.status_code != 200:
    print(f"Ошибка запроса: HTTP {data.status_code}")
    return

  # В переменную soup передаем содержание полученной страницы
  soup = BeautifulSoup(data.content, "lxml")
  # Найдем количество страниц
  # Внизу открытой страницы hh находим количество страниц 250; нажимаем правой кнопкой мыши и далее "Проверить"
  # В открывшемся справа окне, вверху нажимаем кнопку проверить и выбираем количество страниц (250)
  # Находим div class = "pager", и вводим его в attrs
  # Далее надо найти все "span" и взять последний, т.к. там указано 250
  # Далее у последнего span надо получить ссылку "a"
  # В ссылке надо взять "span", и в "span" взять текст
  # Полученное значение переводим в int
  try:
    page_count = int(soup.find("div", attrs={"class":"pager"}).find_all("span",recursive=False)[-1].find("a").find("span").text)
  except Exception as e:
    print("Ошибка при получении количества страниц: ", e)
    return

  for page in range(page_count):
    try:
      # Создаем заголовок и передаем туда случайный заголовок из созданного объекта
      # В запрос передаем страницу
      data = requests.get(
        url=f"{base_url}&page={page}",
        headers = {"user-agent":ua.random}
      )

      if data.status_code != 200:
        print(f"Пропуск страницы {page} из-за ошибки HTTP {data.status_code}")
        continue
      soup = BeautifulSoup(data.content, "lxml")
      # Выбираем на hh любое резюме, нам нужна ссылка "a", у нее нужен атрибут "data-qa"; этот атрибут вставляем в soup
      # Проходим по всем ссылкам
      for a in soup.find_all("a", attrs={"data-qa":"serp-item__title"}):
        # в каждой ссылке "a" нам нужен адрес сайта в "attrs"
        # в адресе сайта нужно всё до знака "?", поэтому разбиваем ссылку по знаку "?" и берем первый элемент
        yield f"https://hh.ru{a.attrs['href'].split('?')[0]}"
    except Exception as e:
      print(f"Ошибка при обработке страницы {page}: {e}")
    time.sleep(1)

In [53]:
# Полученные ссылки соберем в список link_resume
# Для примера спарсил ссылки только с одной страницы
link_resume = [a for a in get_links(base_url)]

In [83]:
len (link_resume)

4978

In [55]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [66]:
# Записываем полученные ссылки в csv-файл
with open(
    f'//content/drive/MyDrive/Practice2/parsinghh/link_resume_{search}.csv',
    'w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    for resume in link_resume:
        writer.writerow([resume])  # Каждая ссылка записывается в новую строку

In [67]:
# Прочитаем ссылки из файла csv и сохраним в переменную link_resume_from_csv
with open(f'/content/drive/MyDrive/Practice2/parsinghh/link_resume_{search}.csv', 'r') as link_csv:
  link_resume_from_csv = [row[0] for row in csv.reader(link_csv)]

In [68]:
link_resume_from_csv

['https://hh.ru/resume/7a03a8c30008087a270039ed1f395254677846',
 'https://hh.ru/resume/970f6fd4000325ead70039ed1f746f69354f78',
 'https://hh.ru/resume/0f705b4e000122a8580039ed1f537742434f49',
 'https://hh.ru/resume/b39abf3e0001f9560d0039ed1f626f4b6d4374',
 'https://hh.ru/resume/9abb08c60003e5fafc0039ed1f723441746669',
 'https://hh.ru/resume/54a173ac0003c7f5fd0039ed1f5a653775304e',
 'https://hh.ru/resume/5660e5f100023409700039ed1f4c4532666565',
 'https://hh.ru/resume/62b20c450003ba2fda0039ed1f644d51466c6a',
 'https://hh.ru/resume/2c21fc3900060f6a270039ed1f464c677a3353',
 'https://hh.ru/resume/16abe7f10000f1e0f60039ed1f354f7457416a',
 'https://hh.ru/resume/904b3f8a00059e30100039ed1f38367130454d',
 'https://hh.ru/resume/816f2ebd0006be82a90039ed1f386969704153',
 'https://hh.ru/resume/27348e8b0003bbff280039ed1f787132735166',
 'https://hh.ru/resume/1db9c37b0005b488b00039ed1f3254336b6355',
 'https://hh.ru/resume/5ebef85b0001f6cece0039ed1f5261636f766b',
 'https://hh.ru/resume/b31261c70005743b4

### Парсинг информации из каждого резюме

In [ ]:
# Переходим по любой ссылке в резюме, с указанной зарплатой
# Смотрим код и копируем class заголовка (для этого нажмем "проверить" и выделим заголовок, т.е. название позиции, например Python developer)
# Аналогично скопируем class зарплаты
# Также ищем class ключевых навыков

In [84]:
# Функция для получения данных резюме
def get_resume(link):

  # Делаем запрос к переданной ссылке
  data = requests.get(
      url = link,
      headers={"user-agent":ua.random}
  )
  if data.status_code != 200:
    return

  soup = BeautifulSoup(data.content, "lxml")

  # Вытащим из ссылки id резюме
  id_link = link.split("/")[-1]

  # Спарсим пол (мужчина / женщина)
  try:
    gender = soup.find("span", attrs={"data-qa": "resume-personal-gender"}).text.replace("\u2009","").replace("\xa0", " ")
  except:
    gender = ""

  # Спарсим возраст
  try:
    age = soup.find("span", attrs={"data-qa": "resume-personal-age"}).text.replace("\u2009","").replace("\xa0", " ")
  except:
    age = ""

  # Спарсим дату рождения
  try:
    date_of_birth = soup.find("span", attrs={"data-qa":"resume-personal-birthday"}).text.replace("\xa0", " ")
  except:
    date_of_birth = ""

  # Спарсим статус поиска работы (Активно ищет работу / Не ищет работу / Рассматривает предложения / и т.д.)
  try:
    job_search_status = soup.find("span", attrs={"data-qa": "job-search-status"}).text
  except:
    job_search_status = ""

    # Спарсим информацию "О себе"
  try:
    about_me = soup.find("div", attrs={"data-qa": "resume-block-skills-content"}).text.strip()
  except:
    about_me = ""

    # Спарсим информацию об образовании
  try:
    # Находим блок образования и извлекаем из него текст
    education_blocks = soup.find_all("div", attrs={"data-qa": "resume-block-education"})
    education = []
    for block in education_blocks:
      education_info = block.text.strip().replace("\n", " ").replace("\xa0", " ")
      education.append(education_info)
  except:
    education = []

    # Спарсим информацию о языках
  try:
     language_blocks = soup.find_all("div", attrs={"data-qa": "resume-block-language-item"})
     language = []
     for block in language_blocks:
         lang_info = block.text.strip().replace("\n", " ").replace("\xa0", " ")
         language.append(lang_info)
  except:
          language = []


  # Спарсим город проживания
  try:
    city_of_residence = soup.find("span", attrs={"data-qa":"resume-personal-address"}).text.replace("\xa0", " ")
  except:
    city_of_residence = ""

  # Спарсим готовность к переезду, командировкам
  try:
    # Находим весь текст в блоке "Готовность к переезду/командировкам"
    movings = soup.find(attrs={"class":"bloko-translate-guard"}).text.replace("\xa0", " ")
    # Разделяем текст на части по запятой
    parts = movings.split(',')
    # Соединяем части, от второй до предпоследней, с запятой-разделителем
    # Первая часть не нужна - это город проживания; последняя - это готовность к командировкам
    # Между первой и последней частями - это готовность к переезду
    # Здесь может быть как одна часть (готов к переезду)
    # Также может быть несколько частей, например готов к переезду (Москва, Россия)
    # В этом примере запятая стоит между "Москва" и "Россия", которая делит нужную фразу на несколько частей
    # Поэтому и собираем несколько частей в одну
    # Получаем готовность к переезду
    moving = ','.join(part for part in parts[1:-1]).strip()
    # Берем последнюю часть, получаем готовность к командировкам
    business_trip = parts[-1].strip()
  except:
    moving = ""
    business_trip = ""

  # Спарсим название позиции
  try:
    name = soup.find(attrs={"class":"resume-block__title-text"}).text
  except:
    name = ""

  # Спарсим зарплату и уберем лишние символы (\u2009, \xa0)
  try:
    salary = soup.find(attrs={"class":"resume-block__salary"}).text.replace("\u2009","").replace("\xa0", " ")
  except:
    salary = ""

  # Спарсим специализации и график работы
  try:
    # Находим все блоки с class "resume-block-container"
    specialized_blocks = soup.find_all(attrs={"class":"resume-block-container"})
    # Проверим каждый блок и найдем блок в котором есть class "resume-block__specialization"
    for block in specialized_blocks:
      if block.find(attrs={"class":"resume-block__specialization"}):
        # Запишем в переменную текст найденного блока
        # Предварительно добавив пробелы перед заглавными буквами и затем удалив лишние пробелы (в начале)
        specialization_schedule = re.sub(r'([А-ЯA-Z])', r' \1', block.text).strip()
  except:
    specialization_schedule = ""

  # Спарсим общее время опыта работы
  try:
    experience_all_period = soup.find(attrs={"class":"resume-block__title-text resume-block__title-text_sub"}).text.replace("\xa0", " ")
  except:
    experience_all_period = ""

  # Спарсим опыт работы
  try:
    # Найдем все блоки с class="resume-block-item-gap" в блоке с "data-qa":"resume-block-experience"
    # В самом первом блоке информация дублируется, поэтому список создадим без самого первого элемента
    blocks_item_gap = soup.find(attrs={"data-qa":"resume-block-experience"}).find_all(attrs={"class":"resume-block-item-gap"})[1:]
    # Создадим паттерн для последующего поиска блоков
    reexp = re.compile('bloko-column.*')
    blocks_all = [] # Сюда будем складывать найденные блоки
    experience = [] # Сюда будем складывать итоговый текст
    # Проходимся по каждому вышенайденному блоку и находим все блоки с атрибутом class по заданному паттерну
    for block_item in blocks_item_gap:
      blocks_all.append (block_item.find_all(class_=reexp)[1:]) # Аналогично нам нужен список без самого первого элемента
    # Проходимся по каждому списку блоков в общем списке blocks_all
    for block in blocks_all:
      # Проходимся по каждому элементу
      for data in block:
        # print(data.text.replace("\xa0", " "))
        # Полученный текст из каждого элемента складываем в итоговый список
        experience.append(data.text.replace("\xa0", " "))
  except:
    experience = ""

  # Спарсим ключевые навыки
  # Находим атрибут, где находятся все ключевые навыки (data-qa = "skills-table); далее находим class для каждого отдельного навыка, оставляем class с text
  try:
    tags = [tag.text for tag in soup.find(attrs={"data-qa":"skills-table"}).find_all(attrs={"class":"bloko-tag__section_text"})]
  except:
    tags = []


  # Собираем все найденные атрибуты в словарь, в конце словаря добавляем url-адрес резюме
  resume = {
      "id":id_link,
      "gender":gender,
      "age":age,
      "date_of_birth":date_of_birth,

      "about_me": about_me,
      "education": education,
      "language": language,

      "job_search_status":job_search_status,
      "city_of_residence":city_of_residence,
      "moving":moving,
      "business_trip":business_trip,
      "name":name,
      "salary":salary,
      "specialization_schedule":specialization_schedule,
      "experience_all_period":experience_all_period,
      "experience":experience,
      "tags":tags,
      "link":link
  }
  return resume

In [85]:
# Проверяем работу на первых 2 ссылках
for a in link_resume[:2]:
  print(get_resume(a))
  time.sleep(1)

{'id': '7a03a8c30008087a270039ed1f395254677846', 'gender': 'Женщина', 'age': '43 года', 'date_of_birth': '20 июля 1980', 'about_me': 'Внимательность, порядочность, аккуратность, целеустремленность, ответственность, пунктуальность, умение работать с людьми разного возраста и статуса. Отсутствие вредных привычек.\nПРОФЕССИОНАЛЬНЫЕ НАВЫКИ: Знание ПК, 1С Торговля и склад, программ Microsoft Office (Word, Excel), Estimate, Гранд-смета и Smeta.Ru (на уровне курсов).\nМОРАЛЬНО-ДЕЛОВЫЕ КАЧЕСТВА: Навыки работы с подрядчиками и заказчиками, ведения переговоров, умение обосновывать и объяснять принятые решения, организаторские способности, умение работать под давлением сроков, умение работать в команде, заинтересованность в повышении квалификации. \nДополнительные навыки:\nАнглийский язык базовый основной, французский язык базовый дополнительный. \nОпыт работы в программном комплексе АРМ «E-PASS» по составлению энергетических паспортов.', 'education': ['Высшее образование2004Московский экономико-

### Парсинг и запись результата в csv файл

In [86]:
# Функция парсинга резюме по ссылкам и записи результатов в файл
def result_resumes(link_resume: list):
  '''
  :link_resume: список ссылок резюме
  '''

  # Создаем файл csv для записи результатов
  with open(
      f'/content/drive/MyDrive/Practice2/parsinghh/result_resumes_{search}.csv',
      'w', newline='', encoding='utf-8') as res_file:
      # Названия столбцов
      fieldnames = ['id', 'gender', 'age',"about_me", "education", "language", 'date_of_birth', 'job_search_status',
                    'city_of_residence', 'moving', 'business_trip', 'name',
                    'salary', 'specialization_schedule',
                    'experience_all_period', 'experience', 'tags', 'link']
      writer = csv.DictWriter(res_file, fieldnames=fieldnames)
      # Записываем заголовки столбцов
      writer.writeheader()

      for link in link_resume:
        resume_data = get_resume(link)
        if resume_data: # При успешном парсинге резюме
          writer.writerow(resume_data)  # Записываем данные в CSV
        print(resume_data)
        time.sleep(1)

In [ ]:
result_resumes(link_resume[:])

{'id': '7a03a8c30008087a270039ed1f395254677846', 'gender': 'Женщина', 'age': '43 года', 'date_of_birth': '20 июля 1980', 'about_me': 'Внимательность, порядочность, аккуратность, целеустремленность, ответственность, пунктуальность, умение работать с людьми разного возраста и статуса. Отсутствие вредных привычек.\nПРОФЕССИОНАЛЬНЫЕ НАВЫКИ: Знание ПК, 1С Торговля и склад, программ Microsoft Office (Word, Excel), Estimate, Гранд-смета и Smeta.Ru (на уровне курсов).\nМОРАЛЬНО-ДЕЛОВЫЕ КАЧЕСТВА: Навыки работы с подрядчиками и заказчиками, ведения переговоров, умение обосновывать и объяснять принятые решения, организаторские способности, умение работать под давлением сроков, умение работать в команде, заинтересованность в повышении квалификации. \nДополнительные навыки:\nАнглийский язык базовый основной, французский язык базовый дополнительный. \nОпыт работы в программном комплексе АРМ «E-PASS» по составлению энергетических паспортов.', 'education': ['Высшее образование2004Московский экономико-

In [ ]:
# Извлекаем информацию из файла с парсингом резюме при помощи pandas
data_res_resume = pd.read_csv(f'/content/drive/MyDrive/Practice2/parsinghh/result_resumes_{search}.csv')

In [ ]:
data_res_resume

,id,gender,age,about_me,education,language,date_of_birth,job_search_status,city_of_residence,moving,business_trip,name,salary,specialization_schedule,experience_all_period,experience,tags,link
0,564d2d4d00090608d70039ed1f497074436c4e,Мужчина,35 лет,NaN,['ОбразованиеСреднее образование'],[],15 марта 1989,Не ищет работу,Новосибирск,"готов к переезду (Россия, Другие регионы)",готов к командировкам,Python junior developer,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 7 месяцев,['Ноябрь 2021 — по настоящее время2 года 7 мес...,"['Амбициозность', 'Python', 'PostgreSQL', 'Git...",https://hh.ru/resume/564d2d4d00090608d70039ed1...
1,0de1b45d0008b61e070039ed1f6d6f4e4e4b43,Мужчина,21 год,Имею опыт разработки в различных сферах на Pyt...,['Высшее образование2025Российский университет...,[],13 сентября 2002,Не ищет работу,Москва,"м. ВДНХ, не готов к переезду",не готов к командировкам,Python-разработчик,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 4 месяца,"['Ноябрь 2021 — Февраль 20242 года 4 месяца', ...","['Python', 'ООП', 'Git', 'PostgreSQL', 'SQL', ...",https://hh.ru/resume/0de1b45d0008b61e070039ed1...
2,5368145600030b31430039ed1f495a32686566,Мужчина,47 лет,В последние годы проходил обучение без возможн...,['Среднее специальное образование1995Ленинград...,[],18 мая 1976,Рассматривает предложения,Ногинск,"готов к переезду (Москва, Россия)",не готов к командировкам,Программист Python,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 8 лет 7 месяцев,['Ноябрь 2015 — по настоящее время8 лет 7 меся...,"['JavaScript', 'Python', 'Linux', 'HTML5', 'SQ...",https://hh.ru/resume/5368145600030b31430039ed1...
3,d0f1b2650008cab3a10039ed1f625374695754,Male,38 years,+7(926)040-34-40\nTelegram: https://t.me/Bruqw...,['Higher education2006Высший институт управлен...,[],16 December 1985,Actively searching for a job,Moscow,willing to relocate,prepared for occasional business trips,Python developer,NaN,"Specializations: Programmer, developer Employm...",Work experience 8 months,"['February 2024 — currently4 months', 'Индивид...","['Python', 'SQL', 'Django Framework', 'English...",https://hh.ru/resume/d0f1b2650008cab3a10039ed1...
4,f241842900057704840039ed1f314d56754168,Мужчина,24 года,"Оптимизация работы ОС Windows, владение аппара...",['Неоконченное высшее образование2022Таганрогс...,[],14 апреля 2000,Не ищет работу,Краснодар,не готов к переезду,не готов к командировкам,Программист Python,500 $ на руки,"Специализации: Программист, разработчик Занято...",Опыт работы 6 лет 5 месяцев,['Январь 2018 — по настоящее время6 лет 5 меся...,"['Настройка ПК', 'Сборка ПК', 'Настройка сетев...",https://hh.ru/resume/f241842900057704840039ed1...
5,b5bd1ef6000917804e0039ed1f483466796348,Мужчина,22 года,"Привет, я - python разработчик. Трудолюбивый и...",['Высшее образование2023Уральский федеральный ...,[],12 сентября 2001,Рассматривает предложения,Екатеринбург,готов к переезду,готов к командировкам,Разработчик Python,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 1 год 5 месяцев,"['Июль 2023 — по настоящее время11 месяцев', '...","['Python', 'SQL', 'Django Framework', 'Git', '...",https://hh.ru/resume/b5bd1ef6000917804e0039ed1...
6,abbff3eb00073b3a000039ed1f5332314a5056,Мужчина,28 лет,Быть программистом – значит решать сложные зад...,['Высшее образование2021Казанский (Приволжский...,[],17 октября 1995,Активно ищет работу,Казань,готов к переезду,готов к командировкам,Python backend разработчик,NaN,"Специализации: Программист, разработчик Занято...",Опыт работы 2 года 5 месяцев,['Январь 2022 — по настоящее время2 года 5 мес...,"['Django', 'Flask', 'Python', 'PostgreSQL', 'S...",https://hh.ru/resume/abbff3eb00073b3a000039ed1...
7,a984f6330008851b520039ed1f6f635430414b,Мужчина,21 год,Желание развиваться профессионально и постоянн...,['ОбразованиеСреднее образование'],[],16 апреля 2003,Активно ищет работу,Новосибирск,готов к переезду,готов к командировкам,Python Developer,NaN,"Спец